# Problem 1.1.1

In [49]:
import xml.etree.ElementTree as ET
import numpy as np

# setup
#folder_path = "data/VOCdevkit/VOC2012/Annotations"
fp = "data/VOCdevkit/VOC2012/Annotations/2007_000027.xml"

def parse_file(fp):    
    # extract xml tree
    tree = ET.parse(fp)
    root = tree.getroot()

    # identify objects
    # extract boundary box data
    output = []
    for obj in root.findall("./object"):
        name = obj.find("name").text
        box = obj.find("bndbox")
        x = int(box.find("xmin").text)
        xmax = int(box.find("xmax").text)
        y = int(box.find("ymin").text)
        ymax = int(box.find("ymax").text)
        
        w = xmax - x
        h = ymax - y
        
        coords = [name, x, y, w, h]
        output.append(coords)

    # put in numpy array format 
    output = np.array(output)
    
    return(output)


# take filepath
# load tree/root
# for object, extract box (class_id, x,y,w,h)
# add to list of bounding boxes
# convert to numpy array
# return array

# Problem 1.1.2

In [50]:
parse_file(fp)

array([['person', '174', '101', '175', '250']], dtype='<U21')

# Problem 1.3

# Problem 2